In [69]:
from sqlite3 import dbapi2 as sqlite3
import datetime
import calendar

In [49]:
def get_next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + datetime.timedelta(days_ahead)

frequencies_counter = {
    0: 4,
    1: 2,
    2: 1
}

frequencies_add_dates = {
    0: 7,
    1: 15,
    2: 31
}

day_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thusday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}

In [6]:
def get_db():
	sqlite_db = sqlite3.connect('data.db')
	sqlite_db.row_factory = sqlite3.Row
	return sqlite_db

In [7]:
def query_db(query, args=(), one=False):
	"""Queries the database and returns a list of dictionaries."""
	cur = get_db().execute(query, args)
	rv = cur.fetchall()
	return (rv[0] if rv else None) if one else rv

In [82]:
db = get_db()
user_id = 1
subscriptions = query_db("select * from subscription where user_id = ?", [user_id])

dates = {}

for subscription in subscriptions:
    days = subscription['days'].split(",")
    
    for day in days:
        frequency_counter, i = frequencies_counter[int(subscription['frequency'])], 1
        
        next_week_day = datetime.datetime.now()
        while i <= frequency_counter:
            next_week_day = get_next_weekday(next_week_day, int(day))
            if next_week_day.date() in dates:
                if subscription['price_id'] not in dates[next_week_day.date()]:
                    dates[next_week_day.date()].append(subscription['price_id'])
            else:
                dates[next_week_day.date()] = None
                dates[next_week_day.date()] = []
                dates[next_week_day.date()].append(subscription['price_id'])
            next_week_day += datetime.timedelta(frequencies_add_dates[int(subscription['frequency'])])
            i += 1
    
sorted_dates = sorted(dates.keys())

data = {}

for date in sorted_dates[:7]:
    month_date = calendar.month_name[date.month] + " " + str(date.day)
    data[month_date] = []
    
    for price_id in dates[date]:
        orders = query_db("select product.img_src, product.name as product_name, price.quantity, seller.name from price, seller, product, subscription where price.product_id = product.id and seller.id = price.seller_id and subscription.price_id = price.id and subscription.user_id = ? and price.id = ?", [user_id, price_id])
        
        for order in orders:
            data[month_date].append(order)

print data

{'April 22': [<sqlite3.Row object at 0x10da9f890>, <sqlite3.Row object at 0x10da9f310>], 'April 21': [<sqlite3.Row object at 0x10da9f8b0>, <sqlite3.Row object at 0x10da9f610>, <sqlite3.Row object at 0x10da9f290>], 'April 26': [<sqlite3.Row object at 0x10da9f5b0>, <sqlite3.Row object at 0x10da9fe50>, <sqlite3.Row object at 0x10da9fe70>, <sqlite3.Row object at 0x10da9fe90>, <sqlite3.Row object at 0x10da9feb0>], 'April 27': [<sqlite3.Row object at 0x10da9ff50>, <sqlite3.Row object at 0x10da9fdf0>, <sqlite3.Row object at 0x10da9fd30>, <sqlite3.Row object at 0x10da9fe30>], 'April 24': [<sqlite3.Row object at 0x10da9fe10>], 'April 25': [<sqlite3.Row object at 0x10da9f0d0>, <sqlite3.Row object at 0x10da9f5d0>], 'May 5': [<sqlite3.Row object at 0x10da9f5f0>]}


In [29]:
print (datetime.date.today()).isoformat()

2017-04-20


In [28]:
print (datetime.date.today() + datetime.timedelta(1)).isoformat()

2017-04-21


In [41]:
def get_next_seven_dates():
    dates = {}
    dates[datetime.date.today()] = None
    dates[datetime.date.today()] = []

    for i in range(1, 7):
        dates[datetime.date.today() + datetime.timedelta(i)] = None
        dates[datetime.date.today() + datetime.timedelta(i)] = []
    
    return dates

In [42]:
dates = get_next_seven_dates()
print len(dates)

7


In [45]:
sorted_dates = sorted(dates.keys())
print sorted_dates

[datetime.date(2017, 4, 20), datetime.date(2017, 4, 21), datetime.date(2017, 4, 22), datetime.date(2017, 4, 23), datetime.date(2017, 4, 24), datetime.date(2017, 4, 25), datetime.date(2017, 4, 26)]
